### Step 1) Find Mean Rf for Each Soil Type (Marine Mud, Holocene Clay/Silt, Glacial Till Clay/Silt (Type A), Glacial Till Clay/Silt (Type B)).

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import xlrd

In [ ]:
# Getting Sheet Names
xls = xlrd.open_workbook(r'Step_1\GEOTEC-DATA.xlsm', on_demand=True)
sheets = xls.sheet_names()

# Removing any sheets that are plots, summary or base and keeping others
sheets = [names for names in sheets if "Project" not in names]
sheets = [names for names in sheets if "base" not in names]
sheets = [names for names in sheets if "soil" not in names]

# sheets

In [ ]:
# Putting all the csv's into one HUUGE DATAFRAME
df0 = pd.DataFrame(columns = ['location', 'description', 'GEOL_STAT', 'top', 'bottom',
       'layer_thickness', 'SOIL', 'stratigraphy', 'perc5.0_qt', 'avg_qt',
       'perc50.0_qt', 'perc95.0_qt', 'perc5.0_fs', 'avg_fs', 'perc50.0_fs',
       'perc95.0_fs', 'avg_su_LB', 'avg_su_BE', 'avg_su_UB', 'avg_phi_eff_LB',
       'avg_phi_eff_BE', 'avg_phi_eff_UB'])


for i in glob.glob('Step_1\Input\*.csv'):
#     print(i)
    # Getting the csv as a dataframe     
    dfi = pd.read_csv(i)
    # Appending the dfi to the df0 and assigning as new df0
    df0 = df0.append(dfi)
    
df0.head()

In [ ]:
# Creating Rf Column
try:
    df0.insert(0,'Rf',np.nan)
except ValueError:
    print('Column already there')
df0.head()

In [ ]:
# Rf = fs/qt
df0['Rf'] = df0['perc50.0_fs'].divide(df0['perc50.0_qt'])
df0.head()

In [ ]:
# Making the average Table for the 4 Soil Types that we need
means = pd.DataFrame(columns = ['description', 'mean_Rf', 'Nk'])
# appending three empty rows
means = means.append(pd.DataFrame([[np.nan] * len(means.columns)], columns=means.columns))
means = means.append(pd.DataFrame([[np.nan] * len(means.columns)], columns=means.columns))
means = means.append(pd.DataFrame([[np.nan] * len(means.columns)], columns=means.columns))
means = means.append(pd.DataFrame([[np.nan] * len(means.columns)], columns=means.columns)).reset_index(drop = True)
means

In [ ]:
# Marine Mud
dfmm = df0.loc[df0['description'] == '26.0_Marine Mud'].dropna(subset = ['Rf'])
# finding the mean
rf_mm = dfmm['Rf'].mean()
# placing in the results table
means['description'][0] = '26.0_Marine Mud'
means['mean_Rf'][0] = rf_mm
means

In [ ]:
# Holocene Clay/Silt
dfhcs = df0.loc[df0['description'] == '27.0_Holocene Clay/Silt'].dropna(subset = ['Rf'])
# finding the mean
rf_hcs = dfhcs['Rf'].mean()
# placing in the results table
means['description'][1] = '27.0_Holocene Clay/Silt'
means['mean_Rf'][1] = rf_hcs
means

In [ ]:
# Type A
dfta = df0.loc[df0['description'] == '28.0_Glacial Till Clay/Silt (Type A)'].dropna(subset = ['Rf'])
# finding the mean
rf_ta = dfta['Rf'].mean()
# placing in the results table
means['description'][2] = '28.0_Glacial Till Clay/Silt (Type A)'
means['mean_Rf'][2] = rf_ta
means

In [ ]:
# Type B
dftb = df0.loc[df0['description'] == '29.0_Glacial Till Clay/Silt (Type B)'].dropna(subset = ['Rf'])
# finding the mean
rf_tb = dftb['Rf'].mean()
# placing in the results table
means['description'][3] = '29.0_Glacial Till Clay/Silt (Type B)'
means['mean_Rf'][3] = rf_tb
means

In [ ]:
# Placing the NKT Coefficients into the table too! From 4.17 Technical Note
means['Nk'][0] = 12.9
means['Nk'][1] = 15.6
means['Nk'][2] = 11.5
means['Nk'][3] = 13.1

means

In [ ]:
# Saving as excel
means.to_excel('Step_1/Output/mean_RF.xlsx')

### Step 2) Parse tables of Geotechnical Expert report into excel. 

In [ ]:
# !pip install tabula-py

In [ ]:
import pandas as pd
import numpy as np 
import tabula
import os
import glob
import xlrd

In [ ]:
xls = xlrd.open_workbook(r'Step_2\\GEOTEC-DATA.xlsm', on_demand=True)
sheets = xls.sheet_names()
# Removing any sheets that are plots, summary or base and keeping others
sheets = [names for names in sheets if "Project" not in names]
sheets = [names for names in sheets if "base" not in names]
sheets = [names for names in sheets if "soil" not in names]
# sheets

In [ ]:
cols = ['Layer', 'Depth From [m]', 'Depth To [m]', 'Soil Description',  \
                   'Soil Type', 'Soil group', 'Consistency', 'Density', 'Eff. Unit Weight [kN/m3] (LB)', \
                  'Eff. Unit Weight [kN/m3] (Char)', 'Eff. Unit Weight [kN/m3] (UB)', 'Eff. Friction Angle (LB)', \
                   'Eff. Friction Angle (Char)', 'Eff. Friction Angle (UB)','Eff. Cohesion [kPa] (LB)', \
                    'Eff. Cohesion [kPa] (Char)','Eff. Cohesion [kPa] (UB)', 'Delta peak (LB)',\
                    'Delta peak (Char)', 'Delta peak (UB)', 'Undr. Shear Strength [kPa] (LB)', \
                    'Undr. Shear Strength [kPa] (Char)', 'Undr. Shear Strength [kPa] (UB)', \
                    'UCS [kPa] (LB)','UCS [kPa] (Char)','UCS [kPa] (UB)']


for i in range(82,124):
    
#     reading
    df = tabula.read_pdf('Step_2\\Input\\EAG-SCH-ENG-REP-GUD-000015 Rev 03 Main Soil and Foundation Expertise Report.pdf', \
                         pages = i, lattice = True)
    
#     getting first table and renaming cols
    data = df[0]
    data.columns = cols
    # Removing the hyphens     
    dash = data['UCS [kPa] (LB)'].iloc[0]
    data = data.replace(dash, np.nan)
    data.to_excel(f'Step_2\\Output\\{sheets[i-82]}_table.xlsx')


for i in range(126,134):
    
#     reading
    df = tabula.read_pdf('Step_2\\Input\\EAG-SCH-ENG-REP-GUD-000015 Rev 03 Main Soil and Foundation Expertise Report.pdf', \
                         pages = i, lattice = True)
    
#     getting first table and renaming cols
    data = df[0]
    data.columns = cols
    # Removing the hyphens     
    dash = data['UCS [kPa] (LB)'].iloc[0]
    data = data.replace(dash, np.nan)
    data.to_excel(f'Step_2\\Output\\{sheets[i-84]}_table.xlsx')


### Step 3) 
#### Find values from PDF cu, gamma, for LB, UB. Calculate for BE and UBk. 
#### Find the overburden pressure for LB, BE, UBk, UB.

In [ ]:
import pandas as pd
import numpy as np 
import glob
import os
import xlrd
import shutil

In [ ]:
# Copying Across the output from Step 2 to Input of step 3
target = 'Step_3\\Input'
for i in glob.glob('Step_2\\Output\\*_table.xlsx'):
    source = i
    # Copying across
    shutil.copy(source, target)
    print(f'{i[-15:-11]} Copied across')

In [ ]:
# Getting Location names
xls = xlrd.open_workbook(r'Step_3\\GEOTEC-DATA.xlsm', on_demand=True)
sheets = xls.sheet_names()
# Removing any sheets that are plots, summary or base and keeping others
sheets = [names for names in sheets if "Project" not in names]
sheets = [names for names in sheets if "soil" not in names]
sheets = [names for names in sheets if "base" not in names]
# sheets

In [ ]:
# TRYING IN BULK
# Getting the Code from a created Soil_Code Excel
Soil_Codes = pd.read_excel(r'Step_3\\Input\\Soil_Codes.xlsx')
Soil_Codes.columns = ['Soil Code', 'Soil Type', 'Density / Consistency', 'Desc']

for sheet in sheets:
    # Reading the pdf table
    pdf_t = pd.read_excel(f'Step_3\\Input\\{sheet}_table.xlsx').iloc[:,1:]
    
    #1
    # Creating the work frame to store final data
    c_first = ['Code', 'Description', 'Depth from [m]', 'Depth to [m]']
    c_add = ['cu from [kPa]', 'cu to [kPa]', "Gamma p [kN/m3]", "s'v0 from [kN/m2]", "s'v0 to [kN/m2]",\
                    'qc [MPa]', 'fs [kPa]']

    # Adding the LB, BE, UB_k and UB to the tiles that change 
    c_second = []
    for item in c_add:
        c_second.append(f'{item} (LB)')
        c_second.append(f'{item} (BE_k)')
        c_second.append(f'{item} (BE)')
        c_second.append(f'{item} (UB_k)')
        c_second.append(f'{item} (UB)')

    c_first.extend(c_second)
    c_titles = c_first

    # Creating the columns and rows and adding titles
    cnames = list(range(0, len(c_titles)))
    rnames = list(range(0,50))
    work = pd.DataFrame(np.nan, index=rnames, columns = c_titles)
    
    #2
    # Getting the Description
    work['Description'] = pdf_t['Layer']
    
    #3
    # Getting the Depth from and to
    work['Depth from [m]'] = pdf_t['Depth From [m]']
    work['Depth to [m]'] = pdf_t['Depth To [m]']
    # Droppin nan here
    work = work.dropna(how = 'all')
    
    
    #4
    # Creating a list of Soil Codes to run through
    for i in range(0,Soil_Codes.shape[0]):
        work.loc[work['Description'] == Soil_Codes['Desc'][i], 'Code'] = Soil_Codes['Soil Code'][i]

    #5
    # Getting the cu from LB, BE_k, UB
    # LB
    work['cu from [kPa] (LB)'] = pdf_t['Undr. Shear Strength [kPa] (LB)']
    work['cu to [kPa] (LB)'] = work['cu from [kPa] (LB)']

    # BE_k
    work['cu from [kPa] (BE_k)'] = pdf_t['Undr. Shear Strength [kPa] (Char)']
    work['cu to [kPa] (BE_k)'] = work['cu from [kPa] (BE_k)']

    # # UB
    work['cu from [kPa] (UB)'] = pdf_t['Undr. Shear Strength [kPa] (UB)']
    work['cu to [kPa] (UB)'] = work['cu from [kPa] (UB)']

    # For case of the Holocene Clay/Silt
    #  LB, UB = (2.30z, 2.80z, 4.25z)
    # Replacing LB and UB with their actual values
    # Depth from
    # LB
    work.loc[work['Description'] == 'Holocene Clay/Silt', 'cu from [kPa] (LB)'] = \
                                    (2.30 * (work.loc[work['Description'] == 'Holocene Clay/Silt', 'Depth from [m]']).astype('float'))  
    work.loc[work['Description'] == 'Holocene Clay/Silt', 'cu to [kPa] (LB)'] = \
                                    (2.30 * (work.loc[work['Description'] == 'Holocene Clay/Silt', 'Depth to [m]']).astype('float'))  

    # BE_k
    work.loc[work['Description'] == 'Holocene Clay/Silt', 'cu from [kPa] (BE_k)'] = \
                                    (2.80 * (work.loc[work['Description'] == 'Holocene Clay/Silt', 'Depth from [m]']).astype('float'))  
    work.loc[work['Description'] == 'Holocene Clay/Silt', 'cu to [kPa] (BE_k)'] = \
                                    (2.80 * (work.loc[work['Description'] == 'Holocene Clay/Silt', 'Depth to [m]']).astype('float'))

    # UB
    work.loc[work['Description'] == 'Holocene Clay/Silt', 'cu from [kPa] (UB)'] = \
                                    (4.25 * (work.loc[work['Description'] == 'Holocene Clay/Silt', 'Depth from [m]']).astype('float'))
    work.loc[work['Description'] == 'Holocene Clay/Silt', 'cu to [kPa] (UB)'] = \
                                    (4.25 * (work.loc[work['Description'] == 'Holocene Clay/Silt', 'Depth to [m]']).astype('float'))

    # Filling BE
    # BE is halfway between UB and LB

    work['cu from [kPa] (BE)'] = (work['cu from [kPa] (LB)'].astype('float') + work['cu from [kPa] (UB)'].astype('float')) / 2

    work['cu to [kPa] (BE)'] = (work['cu to [kPa] (LB)'].astype('float') + work['cu to [kPa] (UB)'].astype('float')) / 2


    # Finding UB_k
    # Check Calc Note
    # cu_UB_k = 2cu_BE - cu_BE_k
    # cu_BE_k defined for Holocene and for others

    # Depth from
    
    work['cu from [kPa] (UB_k)'] = 2 * (work['cu from [kPa] (BE)'].astype('float')) \
                                        - work['cu from [kPa] (BE_k)'].astype('float')

    # Depth to
    work['cu to [kPa] (UB_k)'] = 2 * (work['cu to [kPa] (BE)'].astype('float')) \
                                        - work['cu to [kPa] (BE_k)'].astype('float')
   
    
    
    #5
    # Getting the Gamma p from LB, BE_k, UB
    # LB 
    work['Gamma p [kN/m3] (LB)'] = pdf_t['Eff. Unit Weight [kN/m3] (LB)']

    # BE_k 
    work['Gamma p [kN/m3] (BE_k)'] = pdf_t['Eff. Unit Weight [kN/m3] (Char)']

    # UB
    work['Gamma p [kN/m3] (UB)'] = pdf_t['Eff. Unit Weight [kN/m3] (UB)']

    # Filling BE
    work['Gamma p [kN/m3] (BE)'] = (work['Gamma p [kN/m3] (LB)'].astype('float') + \
                                    work['Gamma p [kN/m3] (UB)'].astype('float')) / 2

    # Finding UB_k
    # Check Calc Note
    # Gamma p_UB_k = 2Gamma p_BE - Gamma p_BE_k
    work['Gamma p [kN/m3] (UB_k)'] = 2*(work['Gamma p [kN/m3] (BE)'].astype('float')) - \
                                work['Gamma p [kN/m3] (BE_k)'].astype('float')
    
    
    #6
    # Getting the s'v0 from LB, BE_k, UB
    # FOR LB
    # LB

    # first row
    # LB
    work["s'v0 from [kN/m2] (LB)"].iloc[0] =  0
    work["s'v0 to [kN/m2] (LB)"].iloc[0] =  work["Gamma p [kN/m3] (LB)"].iloc[0].astype('float') \
                                            * work['Depth to [m]'].iloc[0].astype('float')

    # first row
    # BE_k
    work["s'v0 from [kN/m2] (BE_k)"].iloc[0] = 0
    work["s'v0 to [kN/m2] (BE_k)"].iloc[0] =  work["Gamma p [kN/m3] (BE_k)"].iloc[0].astype('float') \
                                            * work['Depth to [m]'].iloc[0].astype('float')

    # UB
    # first row
    work["s'v0 from [kN/m2] (UB)"].iloc[0] =  0
    work["s'v0 to [kN/m2] (UB)"].iloc[0] =  work["Gamma p [kN/m3] (UB)"].iloc[0].astype('float') \
                                            * work['Depth to [m]'].iloc[0].astype('float')


    for i in range(1,work.shape[0]):
        # LB     
        work["s'v0 from [kN/m2] (LB)"].iloc[i] = work["s'v0 to [kN/m2] (LB)"].iloc[i-1].astype('float')
        work["s'v0 to [kN/m2] (LB)"].iloc[i] = work["s'v0 from [kN/m2] (LB)"].iloc[i].astype('float') + \
                                                work["Gamma p [kN/m3] (LB)"].iloc[i].astype('float') \
                                                * (work['Depth to [m]'].iloc[i].astype('float') - \
                                                   work['Depth to [m]'].iloc[i-1].astype('float'))

        # BE_k     
        work["s'v0 from [kN/m2] (BE_k)"].iloc[i] = work["s'v0 to [kN/m2] (BE_k)"].iloc[i-1].astype('float')
        work["s'v0 to [kN/m2] (BE_k)"].iloc[i] = work["s'v0 from [kN/m2] (BE_k)"].iloc[i].astype('float') + \
                                                work["Gamma p [kN/m3] (BE_k)"].iloc[i].astype('float') \
                                                * (work['Depth to [m]'].iloc[i].astype('float') - \
                                                   work['Depth to [m]'].iloc[i-1].astype('float'))

        # UB    
        work["s'v0 from [kN/m2] (UB)"].iloc[i] = work["s'v0 to [kN/m2] (UB)"].iloc[i-1].astype('float')
        work["s'v0 to [kN/m2] (UB)"].iloc[i] = work["s'v0 from [kN/m2] (UB)"].iloc[i].astype('float') + \
                                                work["Gamma p [kN/m3] (UB)"].iloc[i].astype('float') \
                                                * (work['Depth to [m]'].iloc[i].astype('float') - \
                                                   work['Depth to [m]'].iloc[i-1].astype('float'))

        
    # BE
    work["s'v0 from [kN/m2] (BE)"] = (work["s'v0 from [kN/m2] (LB)"].astype('float') \
                                      + work["s'v0 from [kN/m2] (UB)"].astype('float')) / 2

    work["s'v0 to [kN/m2] (BE)"] = (work["s'v0 to [kN/m2] (LB)"].astype('float') \
                                      + work["s'v0 to [kN/m2] (UB)"].astype('float')) / 2

    # Finding UB_k
    # Check Calc Note
    # s'v0_UB_k = 2s'v0 _BE - s'v0_BE_k
    work["s'v0 from [kN/m2] (UB_k)"] = (2 * work["s'v0 from [kN/m2] (BE)"].astype('float')) \
                                    - work["s'v0 from [kN/m2] (BE_k)"].astype('float')

    work["s'v0 to [kN/m2] (UB_k)"] = (2 * work["s'v0 to [kN/m2] (BE)"].astype('float')) \
                                    - work["s'v0 to [kN/m2] (BE_k)"].astype('float')
    # Saving     
    work.to_excel(f"Step_3\\Output\\{sheet}_sigma_Filled.xlsx", index = False)
    
    print(f'{sheet} processed')
    

Step 4) NKT: Find the qc and fs for LB, BE, UB, UBk using:
	q_c=s_u⋅N_k+σ_v0
	Find fs by *Rf as found in Step i

In [ ]:
# Copying Across the output from Step 1 to Input of Step4 
target = 'Step_4\\Input'
source = 'Step_1\\Output\\mean_RF.xlsx'

# Copying Across 
shutil.copy(source, target)

In [ ]:
# Reading the rf and nk values found previously
rf_nk = pd.read_excel('Step_4/Input/mean_Rf.xlsx')
rf_nk

In [ ]:
# Copying Across the output from Step 3 to Input of step 4
target = 'Step_4\\Input'
for i in glob.glob('Step_3\\Output\\*_sigma_Filled.xlsx'):
    source = i
    # Copying across
    shutil.copy(source, target)
    print(f'{i[14:18]} Copied across')

In [ ]:
# Getting Location names
xls = xlrd.open_workbook(r'Step_4\\GEOTEC-DATA.xlsm', on_demand=True)
sheets = xls.sheet_names()
# Removing any sheets that are plots, summary or base and keeping others
sheets = [names for names in sheets if "Project" not in names]
sheets = [names for names in sheets if "soil" not in names]
sheets = [names for names in sheets if "base" not in names]
# sheets

In [ ]:
for sheet in sheets:
    
    # Opening the loc specific su sheet
    NKTi = pd.read_excel(f'Step_4/Input/{sheet}_sigma_Filled.xlsx')
    
    # Calculating the qc and fs for soilcodes 26 - 29
    soilcodes = [x for x in range(26,30)]
    
    for j, x in enumerate(soilcodes):
        
        rf = rf_nk['mean_Rf'][j]
        nk = rf_nk['Nk'][j]

        # Taking it as avg s'v0 
        # cu doesnt change so just take depth to
        # qc = cu*Nk + s'v0avg
        # LB
        NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (LB)'] = ((NKTi.loc[NKTi['Code'] == x, 'cu to [kPa] (LB)'].astype('float') * nk) + \
                                                        ((NKTi.loc[NKTi['Code'] == x, "s'v0 from [kN/m2] (LB)"].astype('float') + \
                                                        NKTi.loc[NKTi['Code'] == x, "s'v0 to [kN/m2] (LB)"])) / 2) / 1000

        # BE_k
        NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (BE_k)'] = ((NKTi.loc[NKTi['Code'] == x, 'cu to [kPa] (BE_k)'].astype('float') * nk) + \
                                                        ((NKTi.loc[NKTi['Code'] == x, "s'v0 from [kN/m2] (BE_k)"].astype('float') + \
                                                        NKTi.loc[NKTi['Code'] == x, "s'v0 to [kN/m2] (BE_k)"])) / 2) / 1000

        # UB
        NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (UB)'] = ((NKTi.loc[NKTi['Code'] == x, 'cu to [kPa] (UB)'].astype('float') * nk) + \
                                                        ((NKTi.loc[NKTi['Code'] == x, "s'v0 from [kN/m2] (UB)"].astype('float') + \
                                                        NKTi.loc[NKTi['Code'] == x, "s'v0 to [kN/m2] (UB)"])) / 2) / 1000

        # BE
        # BE is LB + UB / 2
        NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (BE)'] = (NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (LB)'].astype('float') \
                                                        + NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (UB)'].astype('float')) / 2

        # Finding UB_k
        # Check Calc Note
        # qc_UB_k = 2qc _BE - qc_BE_k
        NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (UB_k)'] = 2*(NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (BE)'].astype('float')) - \
                                                            NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (BE_k)'].astype('float')


        # fs = qc * Rf
        #  Rf values are taken from all the available CPT for the particular soil type
        # Going from MPa to kPa require * 1000
        # LB
        NKTi.loc[NKTi['Code'] == x, 'fs [kPa] (LB)'] = (NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (LB)'] * rf * 1000)

        # BE_k
        NKTi.loc[NKTi['Code'] == x, 'fs [kPa] (BE_k)'] = (NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (BE_k)'] * rf * 1000)

        # UB
        NKTi.loc[NKTi['Code'] == x, 'fs [kPa] (UB)'] = (NKTi.loc[NKTi['Code'] == x, 'qc [MPa] (UB)'] * rf * 1000)

        # BE
        # BE is LB + UB / 2
        NKTi.loc[NKTi['Code'] == x, 'fs [kPa] (BE)'] = (NKTi.loc[NKTi['Code'] == x, 'fs [kPa] (LB)'] + \
                                                        NKTi.loc[NKTi['Code'] == x, 'fs [kPa] (UB)']) / 2


        # Finding UB_k
        # Check Calc Note
        # qc_UB_k = 2qc _BE - qc_BE_k
        NKTi.loc[NKTi['Code'] == x, 'fs [kPa] (UB_k)'] = 2*(NKTi.loc[NKTi['Code'] == x, 'fs [kPa] (BE)'].astype('float')) - \
                                                            NKTi.loc[NKTi['Code'] == x, 'fs [kPa] (BE_k)'].astype('float')
        
        print(NKTi.iloc[:,18:])
        
        # Saving as sheets
        NKTi.to_excel(f'Step_4\\Output\\{sheet}_NKT_Filled.xlsx')

Step 5) Fill Gaps in CPT with LB, BE, UB, UBk NKT results.

Step 6) Graph 3 qc, fs and Rf side by side. Indicate CPT DOUBLE CHECK THAT IT IS FOR qc and not qt!. 
Indicate DL. Indicate NKT.

Step 7) Back- Calculate the su values for the cohesive layers based on the field qc values and the NKT.
	  Add this to graph in Step vi. 

Step 8) Using the macros bring across Step iv csv's into a final geotec data.